This script evaluates the performance of different graph-aware architectures in a node classification problem. Several datasets are employed paying special attention to the homophily ratio.

In [1]:
import time
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import dgl
import networkx as nx
import torch
import torch.nn as nn


import utils
from gsp_utils.baselines_archs import GCNN_2L
from gsp_utils.baselines_models import NodeClassModel, GF_NodeClassModel
from gsp_utils.data import normalize_gso
from src.arch import GFGCN, GFGCNLayer, GFGCN_noh_Layer, GFGCN_Spows

# SEED = 0
SEED = 15
PATH = 'results/diff_filters/'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(device)

torch.manual_seed(SEED)

cuda:1


In [2]:
def print_full_results(accs, ellapsed_time, datasets, exps):
    mean_accs = accs.mean(axis=2)
    med_accs = np.median(accs, axis=2)
    std_accs = accs.std(axis=2)
    mean_t = ellapsed_time.mean(axis=2)

    for i, dataset_name in enumerate(datasets):
        graph = getattr(dgl.data, dataset_name)(verbose=False)[0]
        edge_hom = dgl.edge_homophily(graph, graph.ndata['label'])

        print(f'{dataset_name} (homophily ratio: {edge_hom:.3f})')
        for j, exp in enumerate(exps):
            print(f'\t- {exp["leg"]}:\tmean: {mean_accs[j,i]:.3f} - std: {std_accs[j,i]:.4f} - med: {med_accs[j,i]:.3f} - time: {mean_t[j,i]:.2f} mins')
        
        print()

def summary_table(accs, datasets, exps, median=False):
    mean_accs = accs.mean(axis=2)
    cols_name = []
    for dataset_name in datasets:
        graph = getattr(dgl.data, dataset_name)(verbose=False)[0]
        edge_hom = dgl.edge_homophily(graph, graph.ndata['label'])
        cols_name.append(f'{dataset_name} ({edge_hom:.2f})')

    index_name = [exp['leg'] for exp in exps]

    return DataFrame(mean_accs, columns=cols_name, index=index_name)


In [3]:
DATASETS = ['TexasDataset',  'WisconsinDataset', 'CornellDataset', 'CiteseerGraphDataset', 'CoraGraphDataset']

EXPS = [{'model': 'Kipf', 'norm': 'both', 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'leg': 'Kipf-A'},
        {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': .01, 'norm': True, 'leg': 'GCN-normA'},
        {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': .01, 'norm': False, 'leg': 'GCN-A'},
        {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': 1, 'norm': True, 'leg': 'GCN-normA-1'},
        {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'GCN-A-1'},
        {'model': 'GFGCN', 'L': 4, 'K': 2, 'h0': .01, 'norm': False, 'leg': 'GCN-A-4-2'},
        {'model': 'H-GFGCN', 'L': 2, 'K': 2, 'norm': True, 'leg': 'GCN-normH'},
        {'model': 'H-GFGCN', 'L': 2, 'K': 2, 'norm': False, 'leg': 'GCN-H'},
        {'model': 'noh-GFGCN', 'L': 2, 'K': 2, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'L': 2, 'K': 2, 'norm': False, 'leg': 'W-GCN-A'},]

## Best params

In [4]:
N_RUNS = 10
ACT = nn.ReLU()  # nn.ReLU()
LAST_ACT = nn.Softmax(dim=1)
LOSS_FN = nn.CrossEntropyLoss() #nn.NLLLoss()

EXPS = [{'model': 'Kipf', 'norm': 'both', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'L': 2, 'hid_dim': 16, 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'L': 2, 'hid_dim': 16, 'leg': 'Kipf-A'},

        {'model': 'GFGCN', 'epochs': 100, 'e_h': 10, 'e_W': 10, 'lr': .05, 'wd': .01, 'drop': 0,
         'hid_dim': 32, 'L': 2, 'K': 3, 'h0': 1, 'norm': True, 'leg': 'A-GCN-normA'},
        {'model': 'GFGCN', 'epochs': 100, 'e_h': 10, 'e_W': 10, 'lr': .05, 'wd': .01, 'drop': 0,
         'hid_dim': 32,'L': 2, 'K': 3, 'h0': 1, 'norm': False, 'leg': 'A-GCN'},

        {'model': 'H-GFGCN', 'epochs': 500, 'e_h': 5, 'e_W': 10, 'lr': .005, 'wd': .01, 'drop': 0,
         'hid_dim': 64,'L': 2, 'K': 2, 'norm': True, 'leg': 'H-GCN-normH'},

        {'model': 'H-GFGCN', 'epochs': 100, 'e_h': 10, 'e_W': 10, 'lr': .05, 'wd': .01, 'drop': 0,
         'hid_dim': 32,'L': 2, 'K': 3, 'norm': False, 'leg': 'H-GCN'},

        {'model': 'noh-GFGCN', 'epochs': 500, 'lr': .05, 'wd': .01, 'drop': 0,
         'hid_dim': 16, 'L': 2, 'K': 3, 'norm': True, 'leg': 'W-GCN-normA'},
         {'model': 'noh-GFGCN', 'epochs': 500, 'lr': .05, 'wd': .01, 'drop': 0,
         'hid_dim': 16, 'L': 2, 'K': 3, 'norm': False, 'leg': 'W-GCN'},]


In [5]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        
        A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device, idx=i)
        N = A.shape[0]
        in_dim = feat.shape[1]
        out_dim = n_class
        
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, exp['hid_dim'], out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=exp['drop'], norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=exp['drop'], diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=exp['drop'], norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=exp['drop'], diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'])
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'],
                                        epochs_h=exp['e_h'], epochs_W=exp['e_W'])
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc1 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val1 = summary_table(accs_best_val, DATASETS, EXPS)

TexasDataset
0: 

0.622 (0.676) - 0.703 (0.703) - 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:43: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(1/S.sum(1))
/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:51: RuntimeWarning: invalid value encountered in matmul
  return D_inv_sqr @ S @ D_inv_sqr


0.108 (0.108) - 0.838 (0.865) - 0.784 (0.892) - 

KeyError: 'h0'

In [ ]:
table_acc_val1

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.648649,0.601961,0.467568,0.2129,0.3161
Kipf-A,0.613514,0.450980,0.362162,0.5888,0.7511
A-GCN-normA,0.156757,0.037255,0.240541,0.7126,0.8084
A-GCN,0.791892,0.823529,0.764865,0.5549,0.5848
H-GCN-normH,0.681081,0.717647,0.597297,0.1877,0.3179
GCN-H,0.816216,0.837255,0.737838,0.5213,0.5624
W-GCN-normA,0.156757,0.037255,0.240541,0.2141,0.3237
W-GCN,0.667568,0.637255,0.551351,0.5370,0.4140


In [ ]:
table_acc1

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.681081,0.652941,0.586486,0.2410,0.3191
Kipf-A,0.629730,0.492157,0.383784,0.5938,0.7607
A-GCN-normA,0.156757,0.037255,0.240541,0.7246,0.8155
A-GCN,0.870270,0.896078,0.840541,0.5694,0.5874
H-GCN-normH,0.759459,0.756863,0.700000,0.2343,0.3195
GCN-H,0.878378,0.901961,0.835135,0.5330,0.5654
W-GCN-normA,0.156757,0.037255,0.240541,0.2494,0.3243
W-GCN,0.724324,0.700000,0.616216,0.5416,0.4140


## Initial Params

In [ ]:
# BEST PARAMETERS
## Reaining params
N_RUNS = 10
N_EPOCHS = 200  # 500
LR = .05
WD = .01
DROPOUT = 0

# BEST PARAMETERS
## Architecture params
N_LAYERS = 2
K = 3
HID_DIM = 50

## Model params
h0 = 1
ACT = nn.ReLU()
LAST_ACT = nn.Softmax(dim=1)
LOSS_FN = nn.CrossEntropyLoss()

EXPS = [{'model': 'Kipf', 'norm': 'both', 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'leg': 'Kipf-A'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': True, 'leg': 'A-GCN-normA'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': False, 'leg': 'A-GCN'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'H-GCN-normH'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'GCN-H'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'W-GCN'},]

In [ ]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        
        A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device, idx=i)
        N = A.shape[0]
        in_dim = feat.shape[1]
        out_dim = n_class
        
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, HID_DIM, out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=DROPOUT, diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=DROPOUT, norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=DROPOUT, diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)

            loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD)
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc2 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val2 = summary_table(accs_best_val, DATASETS, EXPS)

TexasDataset
0: 0.730 (0.730) - 0.730 (0.730) - 0.108 (0.108) - 0.757 (0.838) - 0.838 (0.838) - 0.811 (0.892) - 0.108 (0.108) - 0.757 (0.757) - 
1: 0.649 (0.703) - 0.595 (0.622) - 0.189 (0.189) - 0.838 (0.919) - 0.892 (0.919) - 0.946 (1.000) - 0.189 (0.189) - 0.784 (0.811) - 
2: 0.486 (0.541) - 0.459 (0.514) - 0.216 (0.216) - 0.757 (0.838) - 0.676 (0.730) - 0.784 (0.838) - 0.216 (0.216) - 0.595 (0.622) - 
3: 0.703 (0.703) - 0.622 (0.676) - 0.108 (0.108) - 0.811 (0.892) - 0.649 (0.730) - 0.757 (0.946) - 0.108 (0.108) - 0.703 (0.757) - 
4: 0.622 (0.649) - 0.595 (0.595) - 0.189 (0.189) - 0.838 (0.838) - 0.676 (0.703) - 0.784 (0.865) - 0.189 (0.189) - 0.676 (0.703) - 
5: 0.703 (0.730) - 0.568 (0.595) - 0.189 (0.189) - 0.784 (0.946) - 0.676 (0.811) - 0.784 (0.892) - 0.189 (0.189) - 0.568 (0.649) - 
6: 0.595 (0.622) - 0.568 (0.568) - 0.081 (0.081) - 0.784 (0.892) - 0.622 (0.784) - 0.730 (0.730) - 0.081 (0.081) - 0.622 (0.703) - 
7: 0.703 (0.703) - 0.649 (0.649) - 0.108 (0.108) - 0.730 (0.838

In [ ]:
table_acc_val2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.648649,0.601961,0.467568,0.2129,0.3161
Kipf-A,0.613514,0.450980,0.362162,0.5888,0.7511
A-GCN-normA,0.156757,0.037255,0.240541,0.7126,0.8084
A-GCN,0.791892,0.823529,0.764865,0.5549,0.5848
H-GCN-normH,0.681081,0.717647,0.597297,0.1877,0.3179
GCN-H,0.816216,0.837255,0.737838,0.5213,0.5624
W-GCN-normA,0.156757,0.037255,0.240541,0.2141,0.3237
W-GCN,0.667568,0.637255,0.551351,0.5370,0.4140


In [ ]:
table_acc2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.681081,0.652941,0.586486,0.2410,0.3191
Kipf-A,0.629730,0.492157,0.383784,0.5938,0.7607
A-GCN-normA,0.156757,0.037255,0.240541,0.7246,0.8155
A-GCN,0.870270,0.896078,0.840541,0.5694,0.5874
H-GCN-normH,0.759459,0.756863,0.700000,0.2343,0.3195
GCN-H,0.878378,0.901961,0.835135,0.5330,0.5654
W-GCN-normA,0.156757,0.037255,0.240541,0.2494,0.3243
W-GCN,0.724324,0.700000,0.616216,0.5416,0.4140


## Best params - GF-GNN

In [ ]:
# BEST PARAMETERS
N_RUNS = 10
N_EPOCHS = 100  # 500
EPOCHS_h = 10
EPOCHS_W = 10
LR = .05
WD = .01
DROPOUT = 0
N_LAYERS = 2
K = 3 
HID_DIM = 32
h0 = 1

ACT = nn.ReLU()  # nn.ReLU()
LAST_ACT = nn.Softmax(dim=1)
LOSS_FN = nn.NLLLoss() #nn.CrossEntropyLoss()

EXPS = [{'model': 'Kipf', 'norm': 'both', 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'leg': 'Kipf-A'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': True, 'leg': 'A-GCN-normA'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': False, 'leg': 'A-GCN'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'H-GCN-normH'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'GCN-H'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'W-GCN'},]

In [ ]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        
        A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device, idx=i)
        N = A.shape[0]
        in_dim = feat.shape[1]
        out_dim = n_class
        
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, HID_DIM, out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=DROPOUT, diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=DROPOUT, norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=DROPOUT, diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD)
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD, epochs_h=EPOCHS_h,
                                        epochs_W=EPOCHS_W)
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc2 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val2 = summary_table(accs_best_val, DATASETS, EXPS)

TexasDataset
0: 0.649 (0.649) - 0.676 (0.676) - 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:43: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(1/S.sum(1))
/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:51: RuntimeWarning: invalid value encountered in matmul
  return D_inv_sqr @ S @ D_inv_sqr


0.108 (0.108) - 0.838 (0.865) - 0.649 (0.838) - 0.838 (0.865) - 0.108 (0.108) - 0.730 (0.757) - 
1: 0.595 (0.676) - 0.676 (0.676) - 0.189 (0.189) - 0.892 (0.973) - 0.784 (0.892) - 0.919 (0.973) - 0.189 (0.189) - 0.703 (0.730) - 
2: 0.486 (0.486) - 0.486 (0.486) - 0.216 (0.216) - 0.838 (0.892) - 0.541 (0.541) - 0.784 (0.865) - 0.216 (0.216) - 0.595 (0.595) - 
3: 0.622 (0.622) - 0.649 (0.649) - 0.108 (0.108) - 0.676 (0.919) - 0.595 (0.784) - 0.865 (0.892) - 0.108 (0.108) - 0.730 (0.730) - 
4: 0.595 (0.595) - 0.622 (0.622) - 0.189 (0.189) - 0.838 (0.865) - 0.757 (0.811) - 0.811 (0.892) - 0.189 (0.189) - 0.622 (0.703) - 
5: 0.676 (0.676) - 0.676 (0.676) - 0.189 (0.189) - 0.784 (0.892) - 0.757 (0.838) - 0.757 (0.892) - 0.189 (0.189) - 0.649 (0.649) - 
6: 0.541 (0.568) - 0.595 (0.595) - 0.081 (0.081) - 0.811 (0.946) - 0.649 (0.784) - 0.838 (0.892) - 0.081 (0.081) - 0.676 (0.703) - 
7: 0.622 (0.622) - 0.676 (0.676) - 0.108 (0.108) - 0.811 (0.865) - 0.622 (0.703) - 0.865 (0.865) - 0.108 (0.108

In [ ]:
table_acc_val2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.600000,0.519608,0.459459,0.1985,0.3195
Kipf-A,0.627027,0.449020,0.316216,0.5297,0.7063
A-GCN-normA,0.156757,0.037255,0.240541,0.7113,0.8074
A-GCN,0.800000,0.837255,0.735135,0.5299,0.5760
H-GCN-normH,0.667568,0.652941,0.686486,0.2006,0.3184
GCN-H,0.821622,0.833333,0.764865,0.5922,0.5795
W-GCN-normA,0.156757,0.037255,0.240541,0.2192,0.3259
W-GCN,0.675676,0.605882,0.545946,0.4821,0.3612


In [ ]:
table_acc2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.610811,0.539216,0.516216,0.2345,0.3206
Kipf-A,0.627027,0.466667,0.335135,0.5319,0.7154
A-GCN-normA,0.156757,0.037255,0.240541,0.7185,0.8105
A-GCN,0.891892,0.882353,0.829730,0.5379,0.5788
H-GCN-normH,0.759459,0.701961,0.748649,0.2355,0.3191
GCN-H,0.883784,0.890196,0.827027,0.5939,0.5810
W-GCN-normA,0.156757,0.037255,0.240541,0.2488,0.3274
W-GCN,0.702703,0.666667,0.581081,0.4905,0.3623


## Best params - GF-GNN norm H

In [ ]:
# BEST PARAMETERS
N_RUNS = 10
N_EPOCHS = 500
EPOCHS_h = 5 # 10
EPOCHS_W = 5 # 10
LR = .005
WD = .01
DROPOUT = 0
N_LAYERS = 2
K = 2 
HID_DIM = 50
h0 = 1

ACT = nn.ELU()  # nn.ReLU()
LAST_ACT = nn.Softmax(dim=1)
LOSS_FN = nn.CrossEntropyLoss() #nn.CrossEntropyLoss()

EXPS = [{'model': 'Kipf', 'norm': 'both', 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'leg': 'Kipf-A'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': True, 'leg': 'A-GCN-normA'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': False, 'leg': 'A-GCN'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'H-GCN-normH'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'GCN-H'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'W-GCN'},]

In [ ]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        
        A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device, idx=i)
        N = A.shape[0]
        in_dim = feat.shape[1]
        out_dim = n_class
        
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, HID_DIM, out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=DROPOUT, diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=DROPOUT, norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=DROPOUT, diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD)
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD, epochs_h=EPOCHS_h,
                                        epochs_W=EPOCHS_W)
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc2 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val2 = summary_table(accs_best_val, DATASETS, EXPS)

TexasDataset
0: 0.649 (0.649) - 0.676 (0.676) - 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:43: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(1/S.sum(1))
/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:51: RuntimeWarning: invalid value encountered in matmul
  return D_inv_sqr @ S @ D_inv_sqr


0.108 (0.108) - 0.838 (0.865) - 0.649 (0.838) - 0.838 (0.865) - 0.108 (0.108) - 0.730 (0.757) - 
1: 0.595 (0.676) - 0.676 (0.676) - 0.189 (0.189) - 0.892 (0.973) - 0.784 (0.892) - 0.919 (0.973) - 0.189 (0.189) - 0.703 (0.730) - 
2: 0.486 (0.486) - 0.486 (0.486) - 0.216 (0.216) - 0.838 (0.892) - 0.541 (0.541) - 0.784 (0.865) - 0.216 (0.216) - 0.595 (0.595) - 
3: 0.622 (0.622) - 0.649 (0.649) - 0.108 (0.108) - 0.676 (0.919) - 0.595 (0.784) - 0.865 (0.892) - 0.108 (0.108) - 0.730 (0.730) - 
4: 0.595 (0.595) - 0.622 (0.622) - 0.189 (0.189) - 0.838 (0.865) - 0.757 (0.811) - 0.811 (0.892) - 0.189 (0.189) - 0.622 (0.703) - 
5: 0.676 (0.676) - 0.676 (0.676) - 0.189 (0.189) - 0.784 (0.892) - 0.757 (0.838) - 0.757 (0.892) - 0.189 (0.189) - 0.649 (0.649) - 
6: 0.541 (0.568) - 0.595 (0.595) - 0.081 (0.081) - 0.811 (0.946) - 0.649 (0.784) - 0.838 (0.892) - 0.081 (0.081) - 0.676 (0.703) - 
7: 0.622 (0.622) - 0.676 (0.676) - 0.108 (0.108) - 0.811 (0.865) - 0.622 (0.703) - 0.865 (0.865) - 0.108 (0.108

In [ ]:
table_acc_val2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.648649,0.601961,0.467568,0.2129,0.3161
Kipf-A,0.613514,0.450980,0.362162,0.5888,0.7511
A-GCN-normA,0.156757,0.037255,0.240541,0.7126,0.8084
A-GCN,0.791892,0.823529,0.764865,0.5549,0.5848
H-GCN-normH,0.681081,0.717647,0.597297,0.1877,0.3179
GCN-H,0.816216,0.837255,0.737838,0.5213,0.5624
W-GCN-normA,0.156757,0.037255,0.240541,0.2141,0.3237
W-GCN,0.667568,0.637255,0.551351,0.5370,0.4140


In [ ]:
table_acc2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.681081,0.652941,0.586486,0.2410,0.3191
Kipf-A,0.629730,0.492157,0.383784,0.5938,0.7607
A-GCN-normA,0.156757,0.037255,0.240541,0.7246,0.8155
A-GCN,0.870270,0.896078,0.840541,0.5694,0.5874
H-GCN-normH,0.759459,0.756863,0.700000,0.2343,0.3195
GCN-H,0.878378,0.901961,0.835135,0.5330,0.5654
W-GCN-normA,0.156757,0.037255,0.240541,0.2494,0.3243
W-GCN,0.724324,0.700000,0.616216,0.5416,0.4140
